In [ ]:
# Suprress warnings
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    # Standard imports
    import matplotlib.pyplot as plt
    import numpy as np

    # Drake imports
    from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
    import pydrake.all
    from pydrake.all import (
        RigidTransform, RotationMatrix, RollPitchYaw, RevoluteSpring, LogOutput, SignalLogger, ForceElement, 
        DoorHinge, DoorHingeConfig, PidController, FixedInputPortValue
    )
from paper import Paper

In [ ]:
# Meshcat init
proc, zmq_url, web_url = start_zmq_server_as_subprocess()

In [ ]:
# Constants
PEDESTAL_WIDTH = 0.225
PEDESTAL_HEIGHT = 0.1
PEDESTAL_DEPTH = 0.3

FINGER_RADIUS = 0.01

In [ ]:
def AddPedestal(plant):
    mu = 5.0
    # Parse pedestal model
    parser = pydrake.multibody.parsing.Parser(plant)
    pedestal_instance = parser.AddModelFromFile("pedestal.sdf")
    pedestal_body = plant.GetBodyByName("pedestal_base", pedestal_instance)

    # Weld pedestal to world
    plant.WeldFrames(
        plant.world_frame(),
        plant.GetFrameByName("pedestal_base", pedestal_instance),
        RigidTransform(RotationMatrix(), [0, 0, PEDESTAL_HEIGHT/2])
    )
    
    return pedestal_instance

In [ ]:
def AddPointFinger(plant, init_x=PEDESTAL_WIDTH*0.6, init_z=0):
    mu = 1.0
    radius = FINGER_RADIUS
    finger = plant.AddModelInstance("finger")

    # Add false body at the origin that the finger 
    false_body = plant.AddRigidBody("false_body", finger,
        pydrake.multibody.tree.SpatialInertia(0, [0,0,0], pydrake.multibody.tree.UnitInertia(0,0,0)))

    # Initialize finger body
    finger_body = plant.AddRigidBody("body", finger, 
        pydrake.multibody.tree.SpatialInertia(
            mass=1.0,
            p_PScm_E=np.array([0., 0., 0.]),
            G_SP_E=pydrake.multibody.tree.UnitInertia(1.0, 1.0, 1.0)))

    # Register geometry
    shape = pydrake.geometry.Sphere(radius)
    if plant.geometry_source_is_registered():
        plant.RegisterCollisionGeometry(
            finger_body, RigidTransform(), shape, "body", pydrake.multibody.plant.CoulombFriction(mu, mu))
        plant.RegisterVisualGeometry(finger_body, RigidTransform(), shape, "body", [.9, .5, .5, 1.0])
    
    # Add control joins for x and z movement
    finger_x = plant.AddJoint(pydrake.multibody.tree.PrismaticJoint(
        "finger_x",
        plant.world_frame(),
        plant.GetFrameByName("false_body"), [1, 0, 0], -1, 1))
    plant.AddJointActuator("finger_x", finger_x)
    finger_x.set_default_translation(init_x)
    finger_z = plant.AddJoint(pydrake.multibody.tree.PrismaticJoint(
        "finger_z",
        plant.GetFrameByName("false_body"),
        plant.GetFrameByName("body"), [0, 0, 1], -1, 1))
    finger_z.set_default_translation(init_z)
    plant.AddJointActuator("finger_z", finger_z)

    return finger

In [ ]:
class FingerControl(pydrake.systems.framework.LeafSystem):

    def __init__(self, plant):
        pydrake.systems.framework.LeafSystem.__init__(self)
        self._plant = plant

        self.DeclareVectorOutputPort(
            "finger_actuation", pydrake.systems.framework.BasicVector(2), 
                                    self.CalcOutput)

    def CalcOutput(self, context, output):
        finger_mass = 1
        g = self._plant.gravity_field().gravity_vector()[[0,2]]
        output.SetFromVector(-finger_mass*g + [0, 0.05])

In [ ]:
## Pre-finalize steps
builder = pydrake.systems.framework.DiagramBuilder()

# Add all elements
plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
pedestal_instance = AddPedestal(plant)
# paper_instances = AddPaper(plant, num_links=10, default_joint_angle=-0.001)
paper = Paper(plant, 10)
paper.weld_paper_edge(PEDESTAL_WIDTH, PEDESTAL_HEIGHT)

finger_instance = AddPointFinger(plant)
finger_ctrlr = FingerControl(plant)

In [ ]:
plant.Finalize()

## Post finalize steps
paper.init_ctrlrs(builder)

# Conect finger controller
builder.AddSystem(finger_ctrlr)
builder.Connect(finger_ctrlr.get_output_port(), plant.get_actuation_input_port(finger_instance))

# Visualization and logging
poses_logger = LogOutput(plant.get_state_output_port(), builder)
vis = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder, scene_graph)

# Build diagram and do actions requiring 
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()
paper.connect_ctrlrs(diagram, diagram_context)

In [ ]:
# Finalize simulation and visualization
simulator = pydrake.systems.analysis.Simulator(diagram, diagram_context)
simulator.Initialize()
vis.start_recording()
simulator.AdvanceTo(5)
vis.stop_recording()
vis.publish_recording()

In [ ]:
%matplotlib notebook

plt.plot(poses_logger.sample_times(), poses_logger.data()[0,:], label="Position")
plt.plot(poses_logger.sample_times(), poses_logger.data()[1,:], label="Velocity")
plt.axhline(-np.pi/2, linestyle='--', color='k', label='Setpoint')
plt.legend()
plt.xlabel("Time (s)")
plt.show()